In [34]:
import pandas as pd
import io
import requests
import matplotlib.pyplot as plt

# Replace with the URL you copied
url = 'https://raw.githubusercontent.com/jvilchesf/Learning/main/Andrej_karpathy_videos/Makemore/names.txt'

response = requests.get(url)
words = response.text.splitlines()

In [35]:
chars = sorted(list(set("".join(words))))
stoi = {s:i+1 for i ,s in enumerate(chars)} 
stoi['.']= 0
itos = {i:s for s,i in stoi.items()}

In [36]:
import torch
N = torch.zeros((27,27), dtype = torch.int32)

In [37]:
#Create the training set of biagrams
b = {}
xs = []
ys = []
for w in words[:1]:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        x,y = stoi[ch1], stoi[ch2]
        xs.append(x)
        ys.append(y)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [44]:
import torch.nn.functional as F

#create a generator variable to repeat always the same random numbers
g = torch.Generator().manual_seed(2147483647)
#generate 27 random weights and 27 neurons (weigs x neurons = 27 x 27)  
W = torch.randn((27,27), requires_grad = True, generator = g)

### Forward pass

In [74]:
#Forward pass
#A neuronal network doesn't get the input as a number, it gets it as a one hot encoding format
one_hot_xs = F.one_hot(xs, num_classes=27).float()
#Create the logits, multiplying the one hot encoding by the weights
logits = one_hot_xs @ W 
#logits have to be > 0, so we apply the softmax function
counts = logits.exp()
#Normalization to create probabilities 
probs = counts / counts.sum(1, keepdim = True)
#Loss function
loss = -probs[torch.arange(5),ys].log().mean()

In [75]:
print(loss.item())

3.7091546058654785


In [76]:
#backward pass
W.grad = None
loss.backward()

In [77]:
#Update weights
W.data -= W.grad * 0.1